<a href="https://colab.research.google.com/github/Angry-Jay/ML_TheHiredHand/blob/main/ml-the-hired-hand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Hired Hand

# The project's aim :
To build a **binary classification** model that predicts student placement status based on academic and personal features.

## Source :
https://www.kaggle.com/datasets/ahsan81/job-placement-dataset/data

In [ ]:
# Setting up
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
